# 07. Transformer Decoder Block 구현

## 학습 목표
- Causal Mask (Autoregressive Mask) 이해 및 구현
- Encoder-Decoder vs Decoder-only 구조 차이 이해
- GPT 스타일 Decoder Block 구현

## 관련 면접 질문
- GPT는 왜 Decoder-only 구조인가요?
- Causal Mask가 왜 필요한가요?
- Encoder-Decoder와 Decoder-only의 차이점은?

---
## Pre-Quiz

### Q1. GPT가 Decoder-only를 사용하는 이유는?
```
당신의 답: 
```

### Q2. Causal Mask의 역할은?
```
당신의 답: 
```

### Q3. Encoder-Decoder (T5) vs Decoder-only (GPT)의 용도 차이는?
```
당신의 답: 
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional
import matplotlib.pyplot as plt

torch.manual_seed(42)

# matplotlib 한글 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 이전에 구현한 컴포넌트들 (제공)
def scaled_dot_product_attention(query, key, value, mask=None):
    d_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value), weights

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.0):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        seq_q, seq_k = query.size(1), key.size(1)
        
        Q = self.W_q(query).view(batch_size, seq_q, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, seq_k, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, seq_k, self.num_heads, self.d_k).transpose(1, 2)
        
        Q = Q.contiguous().view(batch_size * self.num_heads, seq_q, self.d_k)
        K = K.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        V = V.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        
        if mask is not None and mask.dim() == 2:
            mask = mask.unsqueeze(0).expand(batch_size * self.num_heads, -1, -1)
        
        output, weights = scaled_dot_product_attention(Q, K, V, mask)
        
        output = output.view(batch_size, self.num_heads, seq_q, self.d_k)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_q, self.d_model)
        
        return self.dropout(self.W_o(output)), weights.view(batch_size, self.num_heads, seq_q, seq_k)

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff=None, dropout=0.1):
        super().__init__()
        if d_ff is None:
            d_ff = d_model * 4
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.dropout(self.fc2(self.dropout(F.gelu(self.fc1(x)))))

---
## 1. Causal Mask 이해

### Autoregressive 모델의 핵심
GPT 같은 모델은 다음 토큰을 예측할 때 **미래 토큰을 보면 안됨**

```
입력: "나는 밥을 ___"
예측: "먹었다"

"밥을" 예측 시 "___"를 보면 치팅!
```

### Causal Mask 구조 (seq_len=4)
```
        Key positions
Query   [0]  [1]  [2]  [3]
[0]      O    X    X    X    ← 0번 위치는 0번만 볼 수 있음
[1]      O    O    X    X    ← 1번 위치는 0,1만 볼 수 있음
[2]      O    O    O    X    ← 2번 위치는 0,1,2만 볼 수 있음
[3]      O    O    O    O    ← 3번 위치는 모두 볼 수 있음

O = attend 가능 (False in mask)
X = attend 불가 (True in mask, -inf로 설정)
```

In [ ]:
# =================================================
# TODO: Causal Mask 생성 함수 구현
# =================================================
#
# 힌트:
# - torch.triu(tensor, diagonal=1)을 사용하면 상삼각 행렬 생성
# - diagonal=1: 대각선 포함하지 않음 (대각선 위만 True)
# - 결과를 bool 타입으로 변환
#
# =================================================

def create_causal_mask(seq_len: int, device: torch.device = None) -> torch.Tensor:
    """
    Causal (autoregressive) mask 생성
    
    Args:
        seq_len: 시퀀스 길이
        device: 텐서가 위치할 device
        
    Returns:
        mask: (seq_len, seq_len) bool tensor
              True인 위치는 attention에서 제외됨
              
    예시 (seq_len=4):
        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]]
    """
    # ============================================
    # 여기에 구현하세요
    # 1. ones 행렬 생성 (seq_len, seq_len)
    # 2. torch.triu로 상삼각 추출 (diagonal=1)
    # 3. bool 타입으로 변환
    # ============================================
    
    pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_causal_mask():
    mask = create_causal_mask(4)
    
    expected = torch.tensor([
        [False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]
    ])
    
    assert torch.equal(mask, expected), f"Mask 오류:\n{mask}"
    print("✅ Causal Mask 테스트 통과!")
    print(f"\nMask (seq_len=4):\n{mask.int()}")

test_causal_mask()

---
## 2. Causal Mask 시각화

In [ ]:
# Causal Mask 시각화
def visualize_causal_mask(seq_len=8):
    mask = create_causal_mask(seq_len)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # 1. Mask 자체
    im1 = axes[0].imshow(mask.int().numpy(), cmap='RdYlGn_r')
    axes[0].set_title('Causal Mask\n(빨강=마스킹, 초록=볼 수 있음)')
    axes[0].set_xlabel('Key Position')
    axes[0].set_ylabel('Query Position')
    plt.colorbar(im1, ax=axes[0])
    
    # 2. Attention 적용 후 (예시)
    torch.manual_seed(42)
    q = torch.randn(1, seq_len, 64)
    k = torch.randn(1, seq_len, 64)
    v = torch.randn(1, seq_len, 64)
    
    _, weights = scaled_dot_product_attention(q, k, v, mask.unsqueeze(0))
    
    im2 = axes[1].imshow(weights[0].numpy(), cmap='Blues')
    axes[1].set_title('Attention Weights with Causal Mask\n(미래 토큰 attention = 0)')
    axes[1].set_xlabel('Key Position')
    axes[1].set_ylabel('Query Position')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()

visualize_causal_mask()

---
## 3. Decoder Block 구현

### GPT 스타일 Decoder (Pre-LN)
```
x → LayerNorm → Masked Self-Attention → + → LayerNorm → FFN → + → output
                                        ↑                     ↑
    ↑___________________________________|   ↑_________________|
              (residual)                      (residual)
```

### Encoder Block과의 차이점
| 항목 | Encoder Block | Decoder Block (GPT) |
|------|--------------|--------------------|
| Mask | 선택적 (패딩용) | 필수 (Causal) |
| Cross-Attention | 없음 | 없음 (Decoder-only) |

In [ ]:
# =================================================
# TODO: Decoder Block 구현 (Pre-LN, GPT 스타일)
# =================================================
#
# Encoder Block과 거의 동일하지만:
# - forward에서 causal mask를 항상 생성하여 적용
# - 추가 mask가 있으면 OR 연산으로 결합
#
# =================================================

class DecoderBlock(nn.Module):
    def __init__(
        self,
        d_model: int,
        num_heads: int,
        d_ff: int = None,
        dropout: float = 0.1
    ):
        super().__init__()
        
        # ============================================
        # 여기에 구현하세요
        # - attention: MultiHeadAttention
        # - ffn: PositionWiseFeedForward
        # - norm1, norm2: LayerNorm
        # - dropout
        # ============================================
        
        pass  # 이 줄을 지우고 구현
    
    def forward(
        self,
        x: torch.Tensor,
        mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        """
        Args:
            x: (batch, seq, d_model)
            mask: 추가 mask (optional, causal mask와 결합)
        Returns:
            output: (batch, seq, d_model)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. causal mask 생성 (x의 seq_len 사용)
        # 2. 추가 mask가 있으면 OR 연산으로 결합
        # 3. Pre-LN + Attention + Residual
        # 4. Pre-LN + FFN + Residual
        # ============================================
        
        pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_decoder_block():
    batch, seq, d_model, num_heads = 2, 10, 64, 8
    
    decoder = DecoderBlock(d_model, num_heads)
    x = torch.randn(batch, seq, d_model)
    
    output = decoder(x)
    
    assert output.shape == x.shape, f"Shape 오류: {output.shape}"
    print("✅ Decoder Block 테스트 통과!")
    print(f"   Input: {x.shape} → Output: {output.shape}")

test_decoder_block()

---
## 4. Transformer Decoder (전체 스택)

In [ ]:
# =================================================
# TODO: Transformer Decoder 구현
# =================================================
#
# 구조:
# token_ids → Embedding → + PE → Dropout → DecoderBlock×N → LayerNorm → Linear → logits
#
# =================================================

class TransformerDecoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        d_model: int,
        num_heads: int,
        num_layers: int,
        d_ff: int = None,
        max_len: int = 512,
        dropout: float = 0.1
    ):
        super().__init__()
        
        # ============================================
        # 여기에 구현하세요
        # - token_embedding: nn.Embedding(vocab_size, d_model)
        # - position_embedding: nn.Embedding(max_len, d_model)
        # - dropout: nn.Dropout
        # - layers: nn.ModuleList of DecoderBlock
        # - norm: nn.LayerNorm (final)
        # - lm_head: nn.Linear(d_model, vocab_size)
        # ============================================
        
        pass  # 이 줄을 지우고 구현
    
    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        Args:
            x: (batch, seq) - token ids
            mask: optional additional mask
        Returns:
            logits: (batch, seq, vocab_size)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. token embedding
        # 2. position ids 생성 (0, 1, 2, ..., seq_len-1)
        # 3. position embedding 더하기
        # 4. dropout
        # 5. decoder blocks 통과
        # 6. final layer norm
        # 7. lm_head로 logits 생성
        # ============================================
        
        pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_transformer_decoder():
    vocab_size = 1000
    batch, seq, d_model, num_heads, num_layers = 2, 10, 64, 8, 4
    
    decoder = TransformerDecoder(
        vocab_size=vocab_size,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers
    )
    
    # 랜덤 토큰 ID 생성
    token_ids = torch.randint(0, vocab_size, (batch, seq))
    
    logits = decoder(token_ids)
    
    assert logits.shape == (batch, seq, vocab_size), f"Shape 오류: {logits.shape}"
    print("✅ Transformer Decoder 테스트 통과!")
    print(f"   Input: {token_ids.shape} → Output: {logits.shape}")
    print(f"   Total parameters: {sum(p.numel() for p in decoder.parameters()):,}")

test_transformer_decoder()

---
## 5. Encoder vs Decoder 비교

In [ ]:
# Encoder vs Decoder Attention 패턴 비교
def compare_encoder_decoder_attention():
    seq_len = 8
    d_model = 64
    
    torch.manual_seed(42)
    x = torch.randn(1, seq_len, d_model)
    
    # Encoder (no mask)
    q, k, v = x, x, x
    _, encoder_weights = scaled_dot_product_attention(
        q, k, v, mask=None
    )
    
    # Decoder (causal mask)
    causal_mask = create_causal_mask(seq_len).unsqueeze(0)
    _, decoder_weights = scaled_dot_product_attention(
        q, k, v, mask=causal_mask
    )
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    im1 = axes[0].imshow(encoder_weights[0].numpy(), cmap='Blues')
    axes[0].set_title('Encoder (Bidirectional)\n모든 위치를 볼 수 있음')
    axes[0].set_xlabel('Key Position')
    axes[0].set_ylabel('Query Position')
    plt.colorbar(im1, ax=axes[0])
    
    im2 = axes[1].imshow(decoder_weights[0].numpy(), cmap='Blues')
    axes[1].set_title('Decoder (Causal/Unidirectional)\n과거 위치만 볼 수 있음')
    axes[1].set_xlabel('Key Position')
    axes[1].set_ylabel('Query Position')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()

compare_encoder_decoder_attention()

---
## Post-Quiz

### Q1. 위 시각화에서 Decoder attention의 상삼각 부분이 0인 이유는?
```
당신의 답: 
```

### Q2. BERT는 왜 Encoder만 사용하나요?
```
당신의 답: 
```

### Q3. T5 같은 Encoder-Decoder 모델에서 Cross-Attention은 어디에 위치하나요?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
def create_causal_mask(seq_len: int, device: torch.device = None) -> torch.Tensor:
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    if device is not None:
        mask = mask.to(device)
    return mask


class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=None, dropout=0.1):
        super().__init__()
        
        self.attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionWiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        seq_len = x.size(1)
        
        # Causal mask 생성
        causal_mask = create_causal_mask(seq_len, x.device)
        
        # 추가 mask와 결합 (OR 연산)
        if mask is not None:
            combined_mask = causal_mask | mask
        else:
            combined_mask = causal_mask
        
        # Pre-LN + Attention + Residual
        normed = self.norm1(x)
        attn_out, _ = self.attention(normed, normed, normed, combined_mask)
        x = x + self.dropout(attn_out)
        
        # Pre-LN + FFN + Residual
        normed = self.norm2(x)
        ffn_out = self.ffn(normed)
        x = x + ffn_out
        
        return x


class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, 
                 d_ff=None, max_len=512, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)
        
        self.layers = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)
    
    def forward(self, x, mask=None):
        batch, seq_len = x.shape
        
        # Embeddings
        tok_emb = self.token_embedding(x)
        pos_ids = torch.arange(seq_len, device=x.device).unsqueeze(0)
        pos_emb = self.position_embedding(pos_ids)
        
        x = self.dropout(tok_emb + pos_emb)
        
        # Decoder blocks
        for layer in self.layers:
            x = layer(x, mask)
        
        # Final norm + projection
        x = self.norm(x)
        logits = self.lm_head(x)
        
        return logits
```

</details>

<details>
<summary>👉 Pre-Quiz 정답</summary>

### Q1. GPT가 Decoder-only를 사용하는 이유는?
**텍스트 생성(다음 토큰 예측)에 최적화**된 구조이기 때문. Encoder의 양방향 컨텍스트는 생성 태스크에 불필요하며, Causal mask로 미래 토큰을 가리면서 autoregressive 생성이 가능.

### Q2. Causal Mask의 역할은?
**미래 토큰에 대한 attention을 차단**하여 모델이 치팅하지 않도록 함. 학습 시 teacher forcing으로 정답을 주지만, 각 위치에서는 이전 토큰만 참고하도록 강제.

### Q3. Encoder-Decoder vs Decoder-only 용도 차이는?
- **Encoder-Decoder (T5, BART)**: 번역, 요약 등 입력→출력 변환 태스크에 적합. 입력 전체를 양방향으로 이해 후 출력 생성.
- **Decoder-only (GPT)**: 텍스트 생성, 대화 등 연속 생성 태스크에 적합. 프롬프트 이어쓰기 형태.

</details>

<details>
<summary>👉 Post-Quiz 정답</summary>

### Q1. 상삼각 부분이 0인 이유
Causal mask가 상삼각 부분(미래 위치)을 -inf로 설정하기 때문. softmax(-inf) = 0이 되어 미래 토큰에 대한 attention weight가 0.

### Q2. BERT가 Encoder만 사용하는 이유
BERT는 **양방향 컨텍스트 이해**가 목적(MLM: Masked Language Modeling). 문장의 앞뒤를 모두 보고 빈칸을 채우므로 causal mask가 오히려 성능 저하. 생성이 아닌 분류/추출 태스크에 최적화.

### Q3. Cross-Attention 위치
Encoder-Decoder 모델의 **Decoder Block 내부**, Self-Attention과 FFN 사이에 위치. Encoder 출력(K, V)과 Decoder 상태(Q)를 연결하여 입력 정보를 참조.

</details>